In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful| packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [2]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config, AdamW
import tensorflow as tf

In [3]:
import csv

# Assuming your CSV file is named 'your_dataset.csv'
csv_file_path = '/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv'

# List to store tuples of (article, highlights)
test_data_list = []

# Read CSV file and extract relevant columns
with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        article = row.get('article', '')
        highlights = row.get('highlights', '')
        test_data_list.append((article, highlights))
        

# Assuming your CSV file is named 'your_dataset.csv'
csv_file_path = '/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv'

# List to store tuples of (article, highlights)
train_data_list = []

# Read CSV file and extract relevant columns
with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        article = row.get('article', '')
        highlights = row.get('highlights', '')
        train_data_list.append((article, highlights))
        
        

# Assuming your CSV file is named 'your_dataset.csv'
csv_file_path = '/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv'

# List to store tuples of (article, highlights)
val_data_list = []

# Read CSV file and extract relevant columns
with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        article = row.get('article', '')
        highlights = row.get('highlights', '')
        val_data_list.append((article, highlights))

In [4]:
import random
# Set the seed for reproducibility
random.seed(42)  # You can choose any seed value

# Calculate 1% of the original data size
one_percent_size = int(0.01 * len(test_data_list))

# Randomly sample 1% of the data
onetest_data_list = random.sample(test_data_list, one_percent_size)
# Calculate 1% of the original data size
one_percent_size = int(0.01 * len(train_data_list))

# Randomly sample 1% of the data
onetrain_data_list = random.sample(train_data_list, one_percent_size)
# Calculate 1% of the original data size
one_percent_size = int(0.01 * len(val_data_list))

# Randomly sample 1% of the data
oneval_data_list = random.sample(val_data_list, one_percent_size)


DATA PREPROCESSING

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and digits
    text = text.translate(str.maketrans('', '', string.punctuation + string.digits))

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:

# Apply preprocessing to your data
ptrain_data_list = [(preprocess_text(article), preprocess_text(highlights)) for article, highlights in onetrain_data_list]
ptest_data_list = [(preprocess_text(article), preprocess_text(highlights)) for article, highlights in onetest_data_list]
pval_data_list = [(preprocess_text(article), preprocess_text(highlights)) for article, highlights in oneval_data_list]

# Print the preprocessed result for the first entry


tokenize and convert to indices

In [7]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define a function to tokenize, convert text to indices, and truncate to the first 1021 tokens
def tokenize_and_convert_to_indices(data_list, max_length=1021):
    tokenized_data_list = []
    for article, highlights in data_list:
        # Tokenize and convert to indices
        article_tokens = tokenizer.encode(article, add_special_tokens=True)
        highlights_tokens = tokenizer.encode(highlights, add_special_tokens=True)

        # Truncate to the first 1021 tokens
        article_tokens = article_tokens[:max_length]
        highlights_tokens = highlights_tokens[:max_length+3]

        # Append to the tokenized_data_list
        tokenized_data_list.append((article_tokens, highlights_tokens))

    return tokenized_data_list

# Apply tokenization and truncation to your datasets
max_length = 1021
tokenized_train_data_list = tokenize_and_convert_to_indices(ptrain_data_list, max_length)
tokenized_test_data_list = tokenize_and_convert_to_indices(ptest_data_list, max_length)
tokenized_val_data_list = tokenize_and_convert_to_indices(pval_data_list, max_length)


Token indices sequence length is longer than the specified maximum sequence length for this model (1046 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
wow = tokenizer._pad_token
print(wow)

None


In [9]:
from transformers import GPT2Tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define a pad token and add it to the tokenizer
pad_token = tokenizer.eos_token
tokenizer.add_tokens([pad_token])

# Define a function to tokenize, convert text to indices, and pad sequences
def tokenize_and_pad(data_list, max_article_length=1021, max_highlights_length=1024):
    tokenized_data_list = []
    for article, highlights in data_list:
        # Tokenize and convert to indices
        article_tokens = tokenizer.encode(article, add_special_tokens=True)
        highlights_tokens = tokenizer.encode(highlights, add_special_tokens=True)

        # Pad sequences to specified lengths
        padded_article_tokens = torch.tensor(article_tokens + [tokenizer.convert_tokens_to_ids(pad_token)] * (max_article_length - len(article_tokens)))
        padded_highlights_tokens = torch.tensor(highlights_tokens + [tokenizer.convert_tokens_to_ids(pad_token)] * (max_highlights_length - len(highlights_tokens)))

        # Append to the tokenized_data_list only if both token lists are not empty
        if len(article_tokens) > 0 and len(highlights_tokens) > 0:
            tokenized_data_list.append((padded_article_tokens, padded_highlights_tokens))

    return tokenized_data_list

# Apply tokenization and padding to your datasets
max_article_length = 1021
max_highlights_length = 1024
tokenized_train_data_list = tokenize_and_pad(ptrain_data_list, max_article_length, max_highlights_length)
tokenized_test_data_list = tokenize_and_pad(ptest_data_list, max_article_length, max_highlights_length)
tokenized_val_data_list = tokenize_and_pad(pval_data_list, max_article_length, max_highlights_length)


Token indices sequence length is longer than the specified maximum sequence length for this model (1046 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
len(tokenized_train_data_list[16][0])

1046

DEFINING TRAIN AND TEST SET

In [11]:
# # Assuming you have tokenized data like tokenized_train_data_list
# # tokenized_train_data_list = [(padded_article_tokens, padded_highlights_tokens), ...]

# # Initialize lists to store input and target ids
# input_ids_train = []
# target_ids_train = []

# # Specify maximum lengths
# max_article_length = 1021
# max_highlights_length = 1024

# # Iterate through the tokenized data
# for padded_article_tokens, padded_highlights_tokens in tokenized_train_data_list:
#     # Truncate article tokens if greater than max_article_length
#     truncated_article_tokens = padded_article_tokens[:max_article_length]

#     # Truncate highlights tokens if greater than max_highlights_length
#     truncated_highlights_tokens = padded_highlights_tokens[:max_highlights_length]

#     # Append truncated article tokens to input_ids_train
#     input_ids_train.append(truncated_article_tokens)

#     # Append truncated highlights tokens to target_ids_train
#     target_ids_train.append(truncated_highlights_tokens)

# # Convert the lists to PyTorch tensors
# input_ids_train = torch.stack(input_ids_train)
# target_ids_train = torch.stack(target_ids_train)


In [12]:
# Initialize lists to store input and target ids
input_ids_val = []
target_ids_val = []

# Specify maximum lengths
max_article_length = 1021
max_highlights_length = 1024

# Iterate through the tokenized data
for padded_article_tokens, padded_highlights_tokens in tokenized_val_data_list:
    # Truncate article tokens if greater than max_article_length
    truncated_article_tokens = padded_article_tokens[:max_article_length]

    # Truncate highlights tokens if greater than max_highlights_length
    truncated_highlights_tokens = padded_highlights_tokens[:max_highlights_length]

    # Append truncated article tokens to input_ids_train
    input_ids_val.append(truncated_article_tokens)

    # Append truncated highlights tokens to target_ids_train
    target_ids_val.append(truncated_highlights_tokens)

# Convert the lists to PyTorch tensors
input_ids_val = torch.stack(input_ids_val)
target_ids_val = torch.stack(target_ids_val)

In [13]:
len(target_ids_train[0])

NameError: name 'target_ids_train' is not defined

TRIAL

In [ ]:

# Tokenize the word "summarize"
tokens = tokenizer.tokenize("summarize")
print(tokens)

In [ ]:
# import torch
# import torch.nn as nn
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, AdamW

# # Load GPT-2 model and tokenizer
# model_name = "gpt2"
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)

# # Define the number of prompts and embedding size
# num_prompts = 4  # "summarize the following text"
# embedding_size = 768

# # Create an embedding layer for soft prompts
# soft_prompt_embeddings = nn.Embedding(num_prompts, embedding_size)

# # Concatenate soft prompt embeddings at the beginning of the input sequence
# class GPT2WithPromptTuning(nn.Module):
#     def __init__(self, gpt2_model, soft_prompt_embeddings):
#         super(GPT2WithPromptTuning, self).__init__()
#         self.gpt2_model = gpt2_model
#         self.soft_prompt_embeddings = soft_prompt_embeddings
    
#     def forward(self, input_ids, soft_prompt_ids):
#         # Get the embeddings for the input_ids from the GPT-2 model
# #         gpt2_outputs = self.gpt2_model(input_ids)
# #         gpt2_embeddings = gpt2_outputs['last_hidden_state']
#         gpt2_embeddings = self.gpt2_model.transformer.wte(input_ids)
#         # Get the embeddings for the soft prompts
#         soft_prompt_embeds = self.soft_prompt_embeddings(soft_prompt_ids)
        
#         # Concatenate the embeddings
#         embeddings = torch.cat([soft_prompt_embeds, gpt2_embeddings], dim=0)
        
#         # Pass the concatenated embeddings through the GPT-2 model
#         outputs = self.gpt2_model(inputs_embeds=embeddings)
        
#         return outputs 

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)

# Define the number of prompts and embedding size
num_prompts = 3  # "summarize the following text"
embedding_size = 768

# Define a specific sentence
sentence = "summarize"
# sum + are + rise
# Tokenize the sentence
input_ids = tokenizer.encode(sentence, return_tensors='pt')

# Get the embeddings for the input_ids from the GPT-2 model
gpt2_embeddings = gpt2_model.transformer.wte(input_ids)

# Create an embedding layer for soft prompts and initialize with the sentence embeddings
soft_prompt_embeddings = nn.Embedding(num_prompts, embedding_size)
soft_prompt_embeddings.weight.data.copy_(gpt2_embeddings.squeeze(0))

# Concatenate soft prompt embeddings at the beginning of the input sequence
class GPT2WithPromptTuning(nn.Module):
    def __init__(self, gpt2_model, soft_prompt_embeddings):
        super(GPT2WithPromptTuning, self).__init__()
        self.gpt2_model = gpt2_model
        self.soft_prompt_embeddings = soft_prompt_embeddings
    
    def forward(self, input_ids, soft_prompt_ids):
        # Get the embeddings for the input_ids from the GPT-2 model
        gpt2_embeddings = self.gpt2_model.transformer.wte(input_ids)
        # Get the embeddings for the soft prompts
        soft_prompt_embeds = self.soft_prompt_embeddings(soft_prompt_ids)
        
        # Concatenate the embeddings
        embeddings = torch.cat([soft_prompt_embeds, gpt2_embeddings], dim=0)
        
        # Pass the concatenated embeddings through the GPT-2 model
        outputs = self.gpt2_model(inputs_embeds=embeddings)
        
        return outputs


In [ ]:
len(target_ids_train)

In [ ]:
# Initialize the model
model = GPT2WithPromptTuning(gpt2_model, soft_prompt_embeddings)

# Freeze GPT-2 model weights
for param in model.gpt2_model.parameters():
    param.requires_grad = False

# Define hyperparameters
batch_size = 8
epochs = 5
learning_rate = 2e-5
gradient_clip_value = 1.0

# Define optimizer and criterion
optimizer = torch.optim.AdamW(model.soft_prompt_embeddings.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

# Example data (replace with your own dataset)
# input_ids = torch.randint(0, 100, (1023,))  # Example input sequence
soft_prompt_ids = torch.tensor([0, 1, 2])
# target_ids = torch.randint(0, 100, (1024,))  # Example input sequence


In [ ]:
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.meteor_score import meteor_score
# from nltk.translate.rouge import Rouge

# Download necessary resources for nltk (run this once)
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download("universal_tagset")
nltk.download("punkt")
nltk.download("stopwords")

In [ ]:
from tqdm import tqdm
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU
model.to(device)

# Define device (assuming you have a CUDA-enabled GPU)

# Move optimizer and criterion to GPU
optimizer = torch.optim.AdamW(model.soft_prompt_embeddings.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=-100).to(device)

# Initialize Weights and Biases
wandb.init(project='SoftPrompt on GPT-2 for Summarization', name='1', config={'learning_rate': learning_rate})

# Training loop
for epoch in range(epochs):
    # Create a tqdm progress bar for the training data
    data_iterator = tqdm(zip(input_ids_train, target_ids_train), desc=f'Epoch {epoch + 1}', total=len(input_ids_train))
    
    for input_ids, target_ids in data_iterator:
        optimizer.zero_grad()

        # Move input and target tensors to GPU
        input_ids, target_ids = input_ids.to(device), target_ids.to(device)
        
        # Assuming you have a soft_prompt_ids for each training instance
        # If not, you might need to modify this part accordingly
        outputs = model(input_ids, soft_prompt_ids.to(device))
        logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state

        loss = criterion(logits, target_ids) 
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_value)

        optimizer.step()

        # Update the progress bar description with the current loss
        data_iterator.set_postfix(loss=loss.item())

#     # Validation loop
    model.eval()
    val_losses = []
    with torch.no_grad():
        for input_ids_val, target_ids_val in zip(input_ids_val, target_ids_val):
            input_ids_val, target_ids_val = input_ids_val.to(device), target_ids_val.to(device)
            outputs_val = model(input_ids_val, soft_prompt_ids.to(device))
            logits_val = outputs_val.logits if hasattr(outputs_val, "logits") else outputs_val.last_hidden_state
            loss_val = criterion(logits_val, target_ids_val)
            val_losses.append(loss_val.item())
            # Convert tensor predictions and references to lists
            predictions = logits_val.argmax(dim=-1).squeeze(0).tolist()
            references = target_ids_val.squeeze(0).tolist()

            # BLEU Score
            bleu_score = sentence_bleu([references], predictions)
            print(f"BLEU Score: {bleu_score}")
    avg_val_loss = sum(val_losses) / len(val_losses)
    wandb.log({"epoch": epoch + 1, "train_loss": loss.item(), "val_loss": avg_val_loss})

    # Set the model back to training mode
#     model.train()

# Close the tqdm progress bar
data_iterator.close()
